##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Оценщики

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/guide/estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/guide/estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Данный раздел переведён с помощью русскоязычного сообщества Tensorflow на общественных началах. Поскольку перевод не является официальным, мы не гарантируем, что он на 100% точен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложения по исправлению перевода, мы будем очень рады увидеть pull request в репозиторий [tensorflow/docs-l10n](https://github.com/tensorflow/docs-l10n) на GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (выполнить перевод или проверить перевод, подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

Этот документ знакомит с `tf.estimator` — высокоуровневым TensorFlow
API. Оценщики включают следующие действия:

* обучение
* оценка
* предсказание
* экспорт для serving

Вы можете использовать созданные нами оценщики или
собственные кастомные оценщики.  Все оценщики - как созданные нами, так и пользовательские - являются
классами, основанными на классе `tf.estimator.Estimator`.

Быстрый пример можно посмотреть в [Учебниках по оценщикам](../../tutorials/estimator/linear.ipynb). Обзор структуры API приведен в [статье](https://arxiv.org/abs/1708.02637).

## Преимущества

Аналогично `tf.keras.Model`, `estimator` это абстракция на уровне модели. `tf.estimator` предоставляет некоторые возможности, которые все еще находятся в стадии разработки для `tf.keras`. Это:

  * Обучение на сервере параметров
  * Полная [TFX](http://tensorflow.org/tfx) интеграция.

## Возможности оценщиков
Оценщики предоставляют следующие преимущества:

* Вы можете запускать модели на основе оценщиков на локальном компьютере или в распределенной многосерверной среде без изменения вашей модели. Кроме того, вы можете запускать модели на основе оценщиков на CPU, GPU, или TPU без изменения исходного кода вашей модели.
* Оценщики обеспечивают безопасный распределенный цикл обучения, который контролирует каким образом и когда:
    * загружаются данные
    * обрабатываются исключения
    * создаются файлы чекпоинтов и восстанавливаются после сбоев
    * сохраняются сводные данные для TensorBoard

При написании приложения с оценщиками, вы должны отделить конвейер входных данных
от модели. Это разделение упрощает эксперименты с различными наборами данных.

## Готовые оценщики

Предварительно созданные оценщики позволяют вам работать на гораздо более высоком концептуальном уровне, чем базовые API TensorFlow. Вам больше не нужно беспокоиться о создании вычислительного графа или сессий, поскольку оценщики делают всю "грязную работу" за вас. Кроме того, оценщики, позволяют вам экспериментировать с различными архитектурами моделей, внося только минимальные изменения в код. `tf.estimator.DNNClassifier`, например, представляет собой заранее подготовленный класс Estimator, который обучает модели классификации на основе полносвязных нейронных сетей прямого распространения.

### Структура программы на готовом оценщике

Создание программы TensorFlow на готовом оценщике обычно состоит из четырех этапов:

#### 1. Напишите одну или несколько функций импорта данных.

Например, вы можете создать одну функцию для импорта тренировочных данных и другую функцию для импорта тестовых данных. Каждая функция импорта данных должна возвращать два объекта:

* словарь, в котором ключами являются имена признаков, а значениями - тензоры (Tensor или SparseTensor), содержащие соответствующие признаку данные
* Tensor, содержащий одну или более меток

Например, следующий код иллюстрирует базовый каркас для входной функции:

```
def input_fn(dataset):
    ...  # манипулирет датасетом, извлекает словарь со свойствами и метку
    return feature_dict, label
```

See [data guide](../../guide/data.md) for details.

#### 2. Определяет столбцы свойств.

Каждый `tf.feature_column` определяет имя признака, его тип и любую предобработку. Например, следующий фрагмент кода создает три столбца признаков содержащих целые числа или значения с плавающей точкой. Первые два столбца признаков просто идентифицируют имя признака и тип. Третий столбец признаков определяет лямбду - программу которая будет вызвана для масштабирования сырых данных:

```
# Определим три столбца с числовыми признаками.
population = tf.feature_column.numeric_column('population')
crime_rate = tf.feature_column.numeric_column('crime_rate')
median_education = tf.feature_column.numeric_column(
  'median_education',
  normalizer_fn=lambda x: x - global_education_mean)
```
Для дополнительной информации см. [учебник по столбцам признаков](https://www.tensorflow.org/tutorials/keras/feature_columns).

#### 3. Создание экземпляра готового оценщика.

Например, вот так можно просто создать экземпляр оценщика `LinearClassifier`:

```
# Создадим экземпляр оценщика, передав столбцы признаков.
estimator = tf.estimator.LinearClassifier(
  feature_columns=[population, crime_rate, median_education])
```
Для дополнительной информации см. [учебник линейной классификации](https://www.tensorflow.org/tutorials/estimator/linear).

#### 4. Вызов метода обучения, оценки или вывода.

Например, все оценщики предоставляют метод `train` который обучает модель.

```
# `input_fn` это функция созданная на шаге 1
estimator.train(input_fn=my_training_set, steps=2000)
```
Вы можете видеть пример этого ниже.

### Преимущества готовых оценщиков

Готовые оценщики включают в себя лучшие практики, предоставляя следующие преимущества:

*   Лучшие практики для определения того, где должны выполняться различные части вычислительного графа, реализуя       стратегии на одной машине или на кластере.
*   Лучшие практики для написания событий (сводок) и универсально полезные сводки.

Если вы не используете готовые оценщики, вам нужно реализовать предыдущие функции самостоятельно.

## Кастомные оценщики

Сердцем каждого оценщика, будь он готовый или кастомный, является его *функция модели*, которая представляет собой метод построения графов для обучения, оценки и прогнозирования. Когда вы используете готовый оценщик, кто-то за вас уже реализовал функцию модели. Если вы пологаетесь на кастомнного оценщика, вам нужно написать функцию модели самостоятельно

## Рекомендуемый процесс работы

1. Предполагая, что существует готовый оценщик, используйте его для посторения своей первой модели и используйте его результаты для определения бейзлайна.
2. Создайте и протестируйте весь конвейер, включая целостность и надежность ваших данных с этим готовым оценщиком.
3. Если имеются другие подходящие готовые оценщики, проведите эксперименты какой из готовых оценщиков дает лучшие результаты.
4. Возможно еще улучшить вашу модель созданием кастомного оценщика.

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

## Создание оценщика из модели Keras

Вы можете конвертировать существующие модели Keras models в оценщики с помощью `tf.keras.estimator.model_to_estimator`. Это позволяет вашей модели Keras
получить сильные стороны оценщиков, такие как, например, распределенное обучение.

Создайте экземпляр модели Keras MobileNet V2 и скомпилируйте модель с оптимизатором, потерями и метриками необходимыми для обучения:

In [ ]:
keras_mobilenet_v2 = tf.keras.applications.MobileNetV2(
    input_shape=(160, 160, 3), include_top=False)
keras_mobilenet_v2.trainable = False

estimator_model = tf.keras.Sequential([
    keras_mobilenet_v2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='softmax')
])

# Компилируем модель
estimator_model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

Создайте `Estimator` из скомпилированной модели Keras. Начальное состояние модели Keras сохранено в созданном `Estimator`:

In [ ]:
est_mobilenet_v2 = tf.keras.estimator.model_to_estimator(keras_model=estimator_model)

Относитесь к полученному `Estimator` так же как к любому другому `Estimator`.

In [ ]:
IMG_SIZE = 160  # Размер всех изображений будет приведен к 160x160

def preprocess(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [ ]:
def train_input_fn(batch_size):
  data = tfds.load('cats_vs_dogs', as_supervised=True)
  train_data = data['train']
  train_data = train_data.map(preprocess).shuffle(500).batch(batch_size)
  return train_data

Для обучения вызовите функцию обучения оценщика:

In [ ]:
est_mobilenet_v2.train(input_fn=lambda: train_input_fn(32), steps=500)

Аналогично, для оценки вызовите функцию оценки оценщика:

In [ ]:
est_mobilenet_v2.evaluate(input_fn=lambda: train_input_fn(32), steps=10)

Дополнительную информацию можно получить в документации по `tf.keras.estimator.model_to_estimator`.